In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 128
epochs = 300
start_epoch = 0
train_batch = 280
test_batch = 280
lr = 0.04
schedule = [75,150,225]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/siamese_aug' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

# iterative training
feedback = 0
# iter_time = [1000, 2000, 2500]

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

# constrastive
thresh = 0.5

# augmentation
blur_prob = 0.1
blog_sig = 0.5
jpg_prob = 0.1

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            while True:
                #keep looping till a different class image is found
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] !=img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
#         img0 = img0.convert("L")
#         img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [8]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [9]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

# Dataset

In [10]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_ = SiameseNetworkDataset(datasets.ImageFolder(train_dir), transform=train_aug, should_invert=False)
train_loader = DataLoader(train_, shuffle=True, num_workers=num_workers, batch_size=train_batch)
val_ = SiameseNetworkDataset(datasets.ImageFolder(val_dir), transform=val_aug, should_invert=False)
val_loader = DataLoader(val_, shuffle=True, num_workers=num_workers, batch_size=test_batch)

# Model

In [11]:
model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.0, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    pt = torch.load(pretrained)['state_dict']
    model.load_state_dict(pt)

In [12]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.17M


# Loss

In [13]:
criterion = ContrastiveLoss(margin=1.0).cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [14]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Train AUROC', 'Valid AUROC'])

In [15]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Train

In [16]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    
    for batch_idx, (inputs0, inputs1, targets) in enumerate(train_loader):
        batch_size = inputs0.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            target_index = targets[targets==0]
            target_index = target_index.long().cuda()
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs0.size(), lam)
            inputs0[target_index, :, bbx1:bbx2, bby1:bby2] = inputs1[target_index, :, bbx1:bbx2, bby1:bby2]
        
        
        outputs0 = model(inputs0)
        outputs1 = model(inputs1)
        
        loss = criterion(outputs0, outputs1, targets)
            
        # compute output
        outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

        # measure accuracy and record loss
        pred = outputs.data
        pred[pred < thresh] = 0.
        pred[pred >= thresh] = 1.
        prec1 = [accuracy_score(targets.data.cpu().numpy(), pred.cpu().numpy())]
        auroc = roc_auc_score(targets.data.cpu().detach().numpy(), outputs.cpu().detach().numpy())
        
        losses.update(loss.data.tolist(), inputs0.size(0))
        top1.update(prec1[0], inputs0.size(0))
        arc.update(auroc, inputs0.size(0))
        

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if batch_idx % 100 == 0:
            print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                 batch=batch_idx+1, size=len(train_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                 batch=batch_idx+1, size=len(train_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs0, inputs1, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()

            # compute output
            outputs0 = model(inputs0)
            outputs1 = model(inputs1)
            loss = criterion(outputs0, outputs1, targets)
            outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

            # measure accuracy and record loss
            pred = outputs.data
            pred[pred < thresh] = 0.
            pred[pred >= thresh] = 1.
            prec1 = [accuracy_score(targets.data.cpu().numpy(), pred.cpu().numpy())]
            auroc = roc_auc_score(targets.data.cpu().detach().numpy(), outputs.cpu().detach().numpy())
            
            losses.update(loss.data.tolist(), inputs0.size(0))
            top1.update(prec1[0], inputs0.size(0))
            arc.update(auroc, inputs0.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
        
    print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                 batch=batch_idx+1, size=len(val_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [19]:
for epoch in range(start_epoch, epochs):    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_loader, model, criterion, epoch, use_cuda)

    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, train_auroc, test_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 300] LR: 0.040000
1/459 | Loss:1.2669 | top1:0.5214 | AUROC:0.5000
101/459 | Loss:0.2999 | top1:0.4994 | AUROC:0.4982
201/459 | Loss:0.2799 | top1:0.5001 | AUROC:0.4998
301/459 | Loss:0.2727 | top1:0.5010 | AUROC:0.5006
401/459 | Loss:0.2690 | top1:0.5011 | AUROC:0.5012
459/459 | Loss:0.2675 | top1:0.5015 | AUROC:0.5016
115/115 | Loss:0.2582 | top1:0.5142 | AUROC:0.5145

Epoch: [2 | 300] LR: 0.042400
1/459 | Loss:0.2609 | top1:0.4929 | AUROC:0.4854
101/459 | Loss:0.2572 | top1:0.5074 | AUROC:0.5083
201/459 | Loss:0.2570 | top1:0.5041 | AUROC:0.5043
301/459 | Loss:0.2565 | top1:0.5062 | AUROC:0.5063
401/459 | Loss:0.2560 | top1:0.5083 | AUROC:0.5083
459/459 | Loss:0.2557 | top1:0.5100 | AUROC:0.5098
115/115 | Loss:0.2337 | top1:0.5988 | AUROC:0.5987

Epoch: [3 | 300] LR: 0.044800
1/459 | Loss:0.2480 | top1:0.5500 | AUROC:0.5500
101/459 | Loss:0.2480 | top1:0.5497 | AUROC:0.5492
201/459 | Loss:0.2435 | top1:0.5679 | AUROC:0.5678
301/459 | Loss:0.2385 | top1:0.5866 | AUROC:0.

101/459 | Loss:0.1793 | top1:0.7186 | AUROC:0.7187
201/459 | Loss:0.1790 | top1:0.7204 | AUROC:0.7203
301/459 | Loss:0.1789 | top1:0.7218 | AUROC:0.7220
401/459 | Loss:0.1780 | top1:0.7235 | AUROC:0.7235
459/459 | Loss:0.1781 | top1:0.7231 | AUROC:0.7231
115/115 | Loss:0.0666 | top1:0.9310 | AUROC:0.9310

Epoch: [23 | 300] LR: 0.092800
1/459 | Loss:0.1687 | top1:0.7464 | AUROC:0.7486
101/459 | Loss:0.1752 | top1:0.7311 | AUROC:0.7312
201/459 | Loss:0.1762 | top1:0.7294 | AUROC:0.7295
301/459 | Loss:0.1754 | top1:0.7299 | AUROC:0.7301
401/459 | Loss:0.1751 | top1:0.7308 | AUROC:0.7309
459/459 | Loss:0.1750 | top1:0.7308 | AUROC:0.7310
115/115 | Loss:0.0701 | top1:0.9329 | AUROC:0.9329

Epoch: [24 | 300] LR: 0.095200
1/459 | Loss:0.1759 | top1:0.7321 | AUROC:0.7332
101/459 | Loss:0.1711 | top1:0.7374 | AUROC:0.7370
201/459 | Loss:0.1713 | top1:0.7389 | AUROC:0.7387
301/459 | Loss:0.1713 | top1:0.7384 | AUROC:0.7383
401/459 | Loss:0.1707 | top1:0.7393 | AUROC:0.7391
459/459 | Loss:0.1706 

301/459 | Loss:0.1169 | top1:0.8319 | AUROC:0.8319
401/459 | Loss:0.1166 | top1:0.8329 | AUROC:0.8329
459/459 | Loss:0.1171 | top1:0.8324 | AUROC:0.8323
115/115 | Loss:0.0375 | top1:0.9543 | AUROC:0.9544

Epoch: [44 | 300] LR: 0.143200
1/459 | Loss:0.1188 | top1:0.8286 | AUROC:0.8251
101/459 | Loss:0.1154 | top1:0.8365 | AUROC:0.8363
201/459 | Loss:0.1156 | top1:0.8360 | AUROC:0.8359
301/459 | Loss:0.1155 | top1:0.8358 | AUROC:0.8357
401/459 | Loss:0.1150 | top1:0.8366 | AUROC:0.8365
459/459 | Loss:0.1151 | top1:0.8363 | AUROC:0.8363
115/115 | Loss:0.0400 | top1:0.9493 | AUROC:0.9492

Epoch: [45 | 300] LR: 0.145600
1/459 | Loss:0.0938 | top1:0.8679 | AUROC:0.8675
101/459 | Loss:0.1144 | top1:0.8385 | AUROC:0.8385
201/459 | Loss:0.1142 | top1:0.8382 | AUROC:0.8383
301/459 | Loss:0.1137 | top1:0.8390 | AUROC:0.8391
401/459 | Loss:0.1138 | top1:0.8386 | AUROC:0.8387
459/459 | Loss:0.1137 | top1:0.8389 | AUROC:0.8390
115/115 | Loss:0.0755 | top1:0.8988 | AUROC:0.8989

Epoch: [46 | 300] LR:

459/459 | Loss:0.0808 | top1:0.8892 | AUROC:0.8892
115/115 | Loss:0.0352 | top1:0.9529 | AUROC:0.9527

Epoch: [65 | 300] LR: 0.159260
1/459 | Loss:0.1025 | top1:0.8464 | AUROC:0.8456
101/459 | Loss:0.0782 | top1:0.8912 | AUROC:0.8913
201/459 | Loss:0.0770 | top1:0.8933 | AUROC:0.8933
301/459 | Loss:0.0778 | top1:0.8919 | AUROC:0.8919
401/459 | Loss:0.0778 | top1:0.8924 | AUROC:0.8925
459/459 | Loss:0.0775 | top1:0.8927 | AUROC:0.8928
115/115 | Loss:0.0402 | top1:0.9446 | AUROC:0.9446

Epoch: [66 | 300] LR: 0.159142
1/459 | Loss:0.0823 | top1:0.8786 | AUROC:0.8785
101/459 | Loss:0.0757 | top1:0.8948 | AUROC:0.8947
201/459 | Loss:0.0753 | top1:0.8954 | AUROC:0.8953
301/459 | Loss:0.0760 | top1:0.8949 | AUROC:0.8948
401/459 | Loss:0.0761 | top1:0.8949 | AUROC:0.8948
459/459 | Loss:0.0755 | top1:0.8960 | AUROC:0.8960
115/115 | Loss:0.0266 | top1:0.9659 | AUROC:0.9659

Epoch: [67 | 300] LR: 0.159015
1/459 | Loss:0.0915 | top1:0.8571 | AUROC:0.8575
101/459 | Loss:0.0766 | top1:0.8953 | AUROC

1/459 | Loss:0.0252 | top1:0.9679 | AUROC:0.9679
101/459 | Loss:0.0186 | top1:0.9769 | AUROC:0.9768
201/459 | Loss:0.0192 | top1:0.9761 | AUROC:0.9761
301/459 | Loss:0.0192 | top1:0.9760 | AUROC:0.9760
401/459 | Loss:0.0190 | top1:0.9762 | AUROC:0.9762
459/459 | Loss:0.0189 | top1:0.9763 | AUROC:0.9763
115/115 | Loss:0.0221 | top1:0.9728 | AUROC:0.9728

Epoch: [87 | 300] LR: 0.015469
1/459 | Loss:0.0206 | top1:0.9750 | AUROC:0.9758
101/459 | Loss:0.0183 | top1:0.9778 | AUROC:0.9779
201/459 | Loss:0.0186 | top1:0.9769 | AUROC:0.9770
301/459 | Loss:0.0188 | top1:0.9766 | AUROC:0.9766
401/459 | Loss:0.0185 | top1:0.9768 | AUROC:0.9768
459/459 | Loss:0.0185 | top1:0.9768 | AUROC:0.9768
115/115 | Loss:0.0189 | top1:0.9757 | AUROC:0.9756

Epoch: [88 | 300] LR: 0.015438
1/459 | Loss:0.0290 | top1:0.9679 | AUROC:0.9679
101/459 | Loss:0.0183 | top1:0.9771 | AUROC:0.9771
201/459 | Loss:0.0180 | top1:0.9775 | AUROC:0.9775
301/459 | Loss:0.0182 | top1:0.9771 | AUROC:0.9772
401/459 | Loss:0.0180 | 

...

115/115 | Loss:0.0171 | top1:0.9792 | AUROC:0.9792

Epoch: [192 | 300] LR: 0.000884
1/459 | Loss:0.0074 | top1:0.9893 | AUROC:0.9894
101/459 | Loss:0.0051 | top1:0.9937 | AUROC:0.9936
201/459 | Loss:0.0051 | top1:0.9938 | AUROC:0.9938
301/459 | Loss:0.0052 | top1:0.9936 | AUROC:0.9936
401/459 | Loss:0.0051 | top1:0.9938 | AUROC:0.9938
459/459 | Loss:0.0050 | top1:0.9939 | AUROC:0.9939
115/115 | Loss:0.0185 | top1:0.9780 | AUROC:0.9780

Epoch: [193 | 300] LR: 0.000875
1/459 | Loss:0.0075 | top1:0.9929 | AUROC:0.9928
101/459 | Loss:0.0055 | top1:0.9935 | AUROC:0.9935
201/459 | Loss:0.0052 | top1:0.9937 | AUROC:0.9937
301/459 | Loss:0.0052 | top1:0.9937 | AUROC:0.9937
401/459 | Loss:0.0052 | top1:0.9936 | AUROC:0.9936
459/459 | Loss:0.0053 | top1:0.9935 | AUROC:0.9935
115/115 | Loss:0.0186 | top1:0.9779 | AUROC:0.9779

Epoch: [194 | 300] LR: 0.000867
1/459 | Loss:0.0114 | top1:0.9893 | AUROC:0.9889
101/459 | Loss:0.0051 | top1:0.9936 | AUROC:0.9937
201/459 | Loss:0.0053 | top1:0.9935 | AU

1/459 | Loss:0.0090 | top1:0.9893 | AUROC:0.9895
101/459 | Loss:0.0050 | top1:0.9938 | AUROC:0.9937
201/459 | Loss:0.0053 | top1:0.9934 | AUROC:0.9934
301/459 | Loss:0.0053 | top1:0.9934 | AUROC:0.9934
401/459 | Loss:0.0053 | top1:0.9934 | AUROC:0.9934
459/459 | Loss:0.0051 | top1:0.9937 | AUROC:0.9937
115/115 | Loss:0.0196 | top1:0.9767 | AUROC:0.9768

Epoch: [214 | 300] LR: 0.000700
1/459 | Loss:0.0030 | top1:0.9964 | AUROC:0.9960
101/459 | Loss:0.0044 | top1:0.9949 | AUROC:0.9949
201/459 | Loss:0.0047 | top1:0.9945 | AUROC:0.9944
301/459 | Loss:0.0048 | top1:0.9943 | AUROC:0.9943
401/459 | Loss:0.0048 | top1:0.9942 | AUROC:0.9942
459/459 | Loss:0.0048 | top1:0.9942 | AUROC:0.9942
115/115 | Loss:0.0200 | top1:0.9763 | AUROC:0.9764

Epoch: [215 | 300] LR: 0.000691
1/459 | Loss:0.0057 | top1:0.9929 | AUROC:0.9932
101/459 | Loss:0.0049 | top1:0.9940 | AUROC:0.9940
201/459 | Loss:0.0049 | top1:0.9940 | AUROC:0.9940
301/459 | Loss:0.0049 | top1:0.9941 | AUROC:0.9941
401/459 | Loss:0.0051 

101/459 | Loss:0.0050 | top1:0.9939 | AUROC:0.9939
201/459 | Loss:0.0052 | top1:0.9937 | AUROC:0.9937
301/459 | Loss:0.0050 | top1:0.9940 | AUROC:0.9940
401/459 | Loss:0.0049 | top1:0.9941 | AUROC:0.9941
459/459 | Loss:0.0050 | top1:0.9940 | AUROC:0.9940
115/115 | Loss:0.0164 | top1:0.9802 | AUROC:0.9802

Epoch: [235 | 300] LR: 0.000053
1/459 | Loss:0.0028 | top1:0.9964 | AUROC:0.9966
101/459 | Loss:0.0047 | top1:0.9942 | AUROC:0.9941
201/459 | Loss:0.0046 | top1:0.9941 | AUROC:0.9941
301/459 | Loss:0.0045 | top1:0.9943 | AUROC:0.9943
401/459 | Loss:0.0048 | top1:0.9940 | AUROC:0.9940
459/459 | Loss:0.0047 | top1:0.9942 | AUROC:0.9942
115/115 | Loss:0.0175 | top1:0.9787 | AUROC:0.9787

Epoch: [236 | 300] LR: 0.000052
1/459 | Loss:0.0005 | top1:1.0000 | AUROC:1.0000
101/459 | Loss:0.0052 | top1:0.9935 | AUROC:0.9936
201/459 | Loss:0.0052 | top1:0.9936 | AUROC:0.9936
301/459 | Loss:0.0054 | top1:0.9934 | AUROC:0.9934
401/459 | Loss:0.0051 | top1:0.9937 | AUROC:0.9937
459/459 | Loss:0.005

201/459 | Loss:0.0049 | top1:0.9940 | AUROC:0.9939
301/459 | Loss:0.0048 | top1:0.9941 | AUROC:0.9940
401/459 | Loss:0.0048 | top1:0.9941 | AUROC:0.9941
459/459 | Loss:0.0048 | top1:0.9941 | AUROC:0.9941
115/115 | Loss:0.0179 | top1:0.9790 | AUROC:0.9790

Epoch: [256 | 300] LR: 0.000037
1/459 | Loss:0.0016 | top1:0.9964 | AUROC:0.9965
101/459 | Loss:0.0042 | top1:0.9948 | AUROC:0.9949
201/459 | Loss:0.0042 | top1:0.9949 | AUROC:0.9949
301/459 | Loss:0.0045 | top1:0.9946 | AUROC:0.9946
401/459 | Loss:0.0047 | top1:0.9944 | AUROC:0.9944
459/459 | Loss:0.0047 | top1:0.9944 | AUROC:0.9944
115/115 | Loss:0.0185 | top1:0.9780 | AUROC:0.9780

Epoch: [257 | 300] LR: 0.000036
1/459 | Loss:0.0004 | top1:1.0000 | AUROC:1.0000
101/459 | Loss:0.0045 | top1:0.9943 | AUROC:0.9942
201/459 | Loss:0.0044 | top1:0.9944 | AUROC:0.9944
301/459 | Loss:0.0045 | top1:0.9943 | AUROC:0.9943
401/459 | Loss:0.0045 | top1:0.9944 | AUROC:0.9944
459/459 | Loss:0.0045 | top1:0.9943 | AUROC:0.9943
115/115 | Loss:0.017

KeyboardInterrupt: 